# Data cleaning

In [91]:
import json
import pandas as pd
import time

In [92]:
d1 = "data/2021_11_11_17_59_Board_94B97E6BA7B0_PowerOnOff_1_u6zwow6t2f25togi_File_0.bmerawdata"
d2 = "data/2021_11_10_16_46_Board_94B97E6BA7B0_PowerOnOff_1_i3pwya3wka1j43oc_File_0.bmerawdata"
d = json.load(open(d2))

In [93]:
data = pd.DataFrame(d["rawDataBody"]["dataBlock"])
data.columns = [item['key'] for item in d["rawDataBody"]["dataColumns"]]
data["date"] = pd.to_datetime(data['real_time_clock'],unit='s')

In [94]:
data.describe()

,sensor_index,sensor_id,timestamp_since_poweron,real_time_clock,temperature,pressure,relative_humidity,resistance_gassensor,heater_profile_step_index,scanning_enabled,label_tag,error_code
count,18400.00000,1.840000e+04,1.840000e+04,1.840000e+04,18400.000000,18400.000000,18400.000000,1.840000e+04,18400.000000,18400.000000,18400.000000,18400.0
mean,3.50000,1.619189e+09,3.659113e+06,1.636566e+09,36.072545,1014.772184,26.117367,4.981060e+06,4.500000,0.980000,1.234402,0.0
std,2.29135,2.842064e+08,2.141301e+06,2.141515e+03,2.010091,0.316654,3.094530,9.435927e+06,2.872359,0.140004,0.835904,0.0
min,0.00000,1.334977e+09,3.274000e+03,1.636563e+09,29.371206,1014.051208,20.457424,3.967762e+04,0.000000,0.000000,0.000000,0.0
25%,1.75000,1.334992e+09,1.805364e+06,1.636565e+09,34.599586,1014.520081,23.558842,1.830533e+05,2.000000,1.000000,0.000000,0.0
50%,3.50000,1.619189e+09,3.659064e+06,1.636566e+09,36.027401,1014.768982,25.777097,4.440590e+05,4.500000,1.000000,1.000000,0.0
75%,5.25000,1.903391e+09,5.514242e+06,1.636568e+09,37.741463,1014.993057,27.875109,4.790294e+06,7.000000,1.000000,2.000000,0.0
max,7.00000,1.903395e+09,7.323845e+06,1.636570e+09,41.912571,1015.629333,39.547005,6.884034e+07,9.000000,1.000000,2.000000,0.0


In [119]:
# fixing messed up labels, more details ask Kuba or Hania
data["fixed_label"] = 0
x = data.label_tag.to_list()
for i in range(1,len(x),1):
    if x[i-1] != x[i]:
        print(i)
data.loc[:4777, "fixed_label"] = 1
data.loc[4777:9200, "fixed_label"] = 2
data.loc[9200:13733, "fixed_label"] = 1
data.loc[13733:, "fixed_label"] = 2

4777
9200
13733


In [140]:
df = data[data.sensor_index == 1]
X = data[["temperature", "pressure", "relative_humidity"]]#, "resistance_gassensor"]]#, "heater_profile_step_index"]]
y = data["fixed_label"]

In [141]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
clf = MLPClassifier(hidden_layer_sizes=(32,32,32,32,32,32), activation='logistic', random_state=100, max_iter=20000).fit(X_train, y_train)

clf.score(X_test, y_test)

0.8582608695652174

# Zapisywanie i wczytywanie modelu

In [142]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8582608695652174
